When you install openpyxl, make sure it's version 3.0.10, or else it won't read the excel file.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels import api as sm
from scipy.optimize import minimize


stock_data = pd.read_excel('monthly-return-capitalization-NEW.xlsx', engine = 'openpyxl')
market_data = pd.read_excel('market-return.xlsx', engine = 'openpyxl')
risk_free_data = pd.read_excel('risk-free-NEW.xlsx', engine = 'openpyxl')

stock_data.head()
# market_data.head()
# risk_free_data.head()


,Header CUSIP -8 Characters,Ticker,PERMCO,Monthly Calendar Date,Monthly Price,Monthly Market Capitalization,Monthly Total Return,Shares Outstanding
0,68389X10,ORCL,8045,2010-01-29,23.0600,115558733.2,-0.058072,5011220
1,68389X10,ORCL,8045,2010-02-26,24.6500,123619750.0,0.068951,5015000
2,68389X10,ORCL,8045,2010-03-31,25.7100,129040829.6,0.043002,5019091
3,68389X10,ORCL,8045,2010-04-30,25.8675,130101186.2,0.008046,5029523
4,68389X10,ORCL,8045,2010-05-28,22.5700,113436820.0,-0.127477,5026000


In [77]:
stock_data['Monthly Calendar Date'] = pd.to_datetime(stock_data['Monthly Calendar Date'])
market_data['Date'] = pd.to_datetime(market_data['Date'])
risk_free_data['Calendar Date'] = pd.to_datetime(risk_free_data['Calendar Date'])

stock_data.rename(columns={'Monthly Calendar Date': 'Date'}, inplace=True)
risk_free_data.rename(columns={'Calendar Date': 'Date'}, inplace=True)


In [ ]:
pivot_df= stock_data.pivot_table(index='Date', columns='Ticker', values='Monthly Total Return')

pivot_df.head(2)
#there are 50 different stocks to analyze

Ticker,AAPL,ABBV,ABT,ADBE,AMD,AMGN,AMZN,AVGO,BAC,CAN,...,PM,TMO,TSLA,TXN,UNH,V,VZ,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-29,-0.088591,NaN,-0.012354,-0.121805,-0.229339,0.033763,-0.067722,-0.049754,0.007968,NaN,...,-0.055613,-0.032292,NaN,-0.132113,0.082677,-0.062086,-0.098769,0.053353,-0.000374,-0.055140
2010-02-26,0.065380,NaN,0.025312,0.072755,0.060322,-0.031977,-0.055897,0.044304,0.097497,NaN,...,0.076247,0.056771,NaN,0.083556,0.026061,0.041173,-0.016655,-0.036631,0.011978,0.015431


In [131]:
#merge all the return datas together
market_data = market_data[['Date', 'Total Market']]
data = pd.merge(pivot_df, market_data, on='Date')
data = pd.merge(data, risk_free_data[['Date', '90 Day Bill Returns']], on='Date')

data.head(2)

,Date,AAPL,ABBV,ABT,ADBE,AMD,AMGN,AMZN,AVGO,BAC,...,TSLA,TXN,UNH,V,VZ,WFC,WMT,XOM,Total Market,90 Day Bill Returns
0,2010-01-29,-0.088591,NaN,-0.012354,-0.121805,-0.229339,0.033763,-0.067722,-0.049754,0.007968,...,NaN,-0.132113,0.082677,-0.062086,-0.098769,0.053353,-0.000374,-0.055140,-0.033607,0.000062
1,2010-02-26,0.065380,NaN,0.025312,0.072755,0.060322,-0.031977,-0.055897,0.044304,0.097497,...,NaN,0.083556,0.026061,0.041173,-0.016655,-0.036631,0.011978,0.015431,0.033732,0.000015


We will set T= 60 and consider dates from January 2017-January 2022. 

In [135]:
data2= data.iloc[84:144]
data2.head(2)

,Date,AAPL,ABBV,ABT,ADBE,AMD,AMGN,AMZN,AVGO,BAC,...,TSLA,TXN,UNH,V,VZ,WFC,WMT,XOM,Total Market,90 Day Bill Returns
84,2017-01-31,0.047747,-0.013898,0.094492,0.101311,-0.085538,0.071609,0.098164,0.128585,0.024434,...,0.178951,0.041853,0.012872,0.060113,-0.071911,0.022138,-0.034433,-0.070574,0.019192,0.000476
85,2017-02-28,0.133743,0.011946,0.079243,0.043747,0.394407,0.134419,0.026182,0.057293,0.090106,...,-0.007701,0.014297,0.020234,0.065237,0.012650,0.034500,0.062781,-0.021713,0.037167,0.000352


,Date,AAPL,ABBV,ABT,ADBE,AMD,AMGN,AMZN,AVGO,BAC,...,TSLA,TXN,UNH,V,VZ,WFC,WMT,XOM,Total Market,90 Day Bill Returns
84,2017-01-31,0.047747,-0.013898,0.094492,0.101311,-0.085538,0.071609,0.098164,0.128585,0.024434,...,0.178951,0.041853,0.012872,0.060113,-0.071911,0.022138,-0.034433,-0.070574,0.019192,0.000476
85,2017-02-28,0.133743,0.011946,0.079243,0.043747,0.394407,0.134419,0.026182,0.057293,0.090106,...,-0.007701,0.014297,0.020234,0.065237,0.012650,0.034500,0.062781,-0.021713,0.037167,0.000352
86,2017-03-31,0.048690,0.053687,-0.014862,0.099628,0.006224,-0.070583,0.049110,0.042831,-0.041354,...,0.113245,0.051423,-0.004593,0.010575,-0.017731,-0.038355,0.023638,0.008485,0.000813,0.000384
87,2017-04-28,-0.000070,0.022062,-0.011363,0.027741,-0.085911,-0.004571,0.043371,0.008449,-0.010598,...,0.128530,-0.011048,0.066276,0.026443,-0.047027,-0.032699,0.043008,-0.004390,0.010580,0.000617
88,2017-05-31,0.067770,0.001213,0.046288,0.060715,-0.158647,-0.042640,0.075276,0.084552,-0.036632,...,0.085777,0.041803,0.001715,0.045837,0.015901,-0.043568,0.052443,-0.004816,0.010208,0.000588
89,2017-06-30,-0.057214,0.098304,0.064608,-0.002961,0.115282,0.109443,-0.026764,-0.022679,0.082552,...,0.060409,-0.067402,0.062855,-0.015226,-0.042453,0.083496,-0.037150,0.002857,0.009267,0.000873
90,2017-07-31,0.032704,-0.027273,0.017262,0.035704,0.090545,0.013238,0.020434,0.058399,-0.005771,...,-0.105473,0.064377,0.034462,0.061634,0.098125,-0.026530,0.056950,-0.008547,0.018719,0.000917
91,2017-08-31,0.107142,0.077099,0.035787,0.059185,-0.044820,0.025570,-0.007269,0.021933,-0.004556,...,0.100257,0.017695,0.036964,0.041436,-0.008884,-0.046321,-0.017904,-0.037053,0.001627,0.000978
92,2017-09-29,-0.060244,0.180080,0.047507,-0.038541,-0.019231,0.048827,-0.019631,-0.033878,0.060695,...,-0.041585,0.082347,-0.011608,0.016615,0.031686,0.079890,0.000897,0.074021,0.024602,0.000802
93,2017-10-31,0.096808,0.022753,0.021227,0.174152,-0.138431,-0.060231,0.149717,0.088109,0.080900,...,-0.028056,0.085610,0.073372,0.045040,-0.021042,0.017951,0.117353,0.016711,0.021648,0.000939


In [ ]:
average_returns = data.mean()


average_returns.head

Date                   2016-07-15 00:46:09.230769152
AAPL                                        0.022618
ABBV                                        0.018982
ABT                                          0.01263
ADBE                                        0.017189
AMD                                         0.025215
AMGN                                        0.013776
AMZN                                        0.020023
AVGO                                        0.027342
BAC                                         0.010597
CAN                                          0.04163
CAT                                         0.014983
CMCSA                                       0.013003
COP                                         0.014619
COST                                        0.016569
CRM                                         0.016463
CSCO                                        0.009346
CVX                                         0.011378
DHR                                         0.

In [135]:
def estimate_betas(data):
    """Estimate betas using a single-factor model"""
    betas = {}
    for ticker in data['Ticker'].unique():
        stock_returns = data[data['Ticker'] == ticker]['Stock Return'].dropna()
        market_returns = data['Market Return'].loc[stock_returns.index]
        cov_matrix = np.cov(stock_returns, market_returns)
        beta = cov_matrix[0, 1] / cov_matrix[1, 1]
        betas[ticker] = beta
    return betas

betas = estimate_betas(data)

In [8]:
from sklearn.linear_model import LinearRegression

def calculate_betas(data, stock_columns, market_column):
    betas = {}
    for stock in stock_columns:
        model = LinearRegression()
        model.fit(data[market_column].values.reshape(-1, 1), data[stock].values)
        betas[stock] = model.coef_[0]
    return betas

# Calculate betas for each stock
betas = calculate_betas(data, stock_data.columns, 'Market_Return')

# Compute the market variance
market_variance = np.var(data['Market_Return'])

# Estimate the covariance matrix using the single-factor model
cov_matrix = np.outer(np.array(list(betas.values())), np.array(list(betas.values()))) * market_variance
cov_matrix = pd.DataFrame(cov_matrix, index=stock_data.columns, columns=stock_data.columns)


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.optimize import minimize

# # Load the data
# stock_data = pd.read_excel('monthly-return-capitalization-NEW.xlsx')
# market_data = pd.read_excel('market-return.xlsx')
# risk_free_data = pd.read_excel('risk-free-NEW.xlsx')

# # Data Preprocessing
# stock_data['Date'] = pd.to_datetime(stock_data['Monthly Calendar Date'])
# market_data['Date'] = pd.to_datetime(market_data['Date'])
# risk_free_data['Date'] = pd.to_datetime(risk_free_data['Calendar Date'])

# # Select relevant columns
# stock_data = stock_data[['Date', 'Ticker', 'Monthly Total Return', 'Monthly Market Capitalization']]
# market_data = market_data[['Date', 'Total Market']]

# # Merging stock data with market data to align on dates
# data = pd.merge(stock_data, market_data, on='Date')

# # Calculate the market return (monthly return)
# market_data['Market Return'] = market_data['Total Market'].pct_change()

# # Calculate stock returns (monthly returns)
# data['Stock Return'] = data['Monthly Total Return'].pct_change()

# # Merge risk-free data
# data = pd.merge(data, risk_free_data[['Date', '90 Day Bill Returns']], on='Date')

# # Step 1: Covariance Estimation using a Single-Factor Model
# def estimate_betas(data):
#     """Estimate betas using a single-factor model"""
#     betas = {}
#     for ticker in data['Ticker'].unique():
#         stock_returns = data[data['Ticker'] == ticker]['Stock Return'].dropna()
#         market_returns = data['Market Return'].loc[stock_returns.index]
#         cov_matrix = np.cov(stock_returns, market_returns)
#         beta = cov_matrix[0, 1] / cov_matrix[1, 1]
#         betas[ticker] = beta
#     return betas

# betas = estimate_betas(data)

# # Step 2: Portfolio Construction - Various Approaches

# # Helper functions for portfolio construction
# def minimum_risk_portfolio(cov_matrix):
#     """Calculate minimum risk portfolio weights"""
#     inv_cov_matrix = np.linalg.inv(cov_matrix)
#     ones = np.ones(len(inv_cov_matrix))
#     w = inv_cov_matrix @ ones
#     w /= np.sum(w)  # Normalize the weights
#     return w

# def maximum_diversification_portfolio(cov_matrix, betas):
#     """Calculate maximum diversification portfolio weights"""
#     volatilities = np.sqrt(np.diagonal(cov_matrix))
#     diversifications = volatilities / np.dot(betas, volatilities)
#     weights = diversifications / np.sum(diversifications)
#     return weights

# def risk_parity_portfolio(cov_matrix):
#     """Calculate risk parity portfolio weights"""
#     n = len(cov_matrix)
#     def objective(w):
#         portfolio_volatility = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))
#         risk_contribs = np.dot(cov_matrix, w) * w
#         risk_parity = np.sum(np.abs(risk_contribs - portfolio_volatility / n))
#         return risk_parity
#     initial_guess = np.ones(n) / n
#     result = minimize(objective, initial_guess, constraints={'type': 'eq', 'fun': lambda w: np.sum(w) - 1})
#     return result.x

# def value_weighted_portfolio(data):
#     """Calculate value-weighted portfolio"""
#     market_caps = data.groupby('Ticker')['Monthly Market Capitalization'].mean()
#     total_market_cap = market_caps.sum()
#     weights = market_caps / total_market_cap
#     return weights

# def equally_weighted_portfolio(n):
#     """Calculate equally weighted portfolio"""
#     return np.ones(n) / n

# # Step 3: Backtesting and Out-of-Sample Evaluation
# def backtest_portfolio(data, betas, portfolio_func, lookback=60, rebalance_months=1):
#     """Backtest the portfolio strategy"""
#     returns = []
#     portfolio_weights = []

#     for i in range(lookback, len(data)):
#         train_data = data.iloc[i - lookback:i]
        
#         # Covariance estimation
#         cov_matrix = train_data[['Stock Return', 'Market Return']].cov().values
        
#         # Get the portfolio weights using the chosen method
#         weights = portfolio_func(cov_matrix)
#         portfolio_weights.append(weights)

#         # Calculate portfolio return for next month
#         next_month_return = data.iloc[i]['Stock Return']
#         portfolio_return = np.dot(weights, next_month_return)
#         returns.append(portfolio_return)

#     return np.array(returns), np.array(portfolio_weights)

# # Step 4: Performance Evaluation (Cumulative Returns, Sharpe Ratio, etc.)
# def evaluate_performance(returns, risk_free_rate):
#     """Evaluate the performance of a portfolio"""
#     excess_returns = returns - risk_free_rate
#     cumulative_returns = np.cumprod(1 + excess_returns) - 1
#     sharpe_ratio = np.mean(excess_returns) / np.std(excess_returns)
#     max_drawdown = np.min(cumulative_returns)
#     return cumulative_returns, sharpe_ratio, max_drawdown

# # Step 5: Visualization (Cumulative Returns)
# def plot_cumulative_returns(cumulative_returns, labels):
#     """Plot cumulative returns for different strategies"""
#     plt.figure(figsize=(10, 6))
#     for i, cr in enumerate(cumulative_returns):
#         plt.plot(cr, label=labels[i])
#     plt.title('Cumulative Returns Comparison')
#     plt.xlabel('Time')
#     plt.ylabel('Cumulative Return')
#     plt.legend()
#     plt.show()

# # Main Execution

# # Backtest portfolios
# min_risk_returns, min_risk_weights = backtest_portfolio(data, betas, minimum_risk_portfolio)
# max_div_returns, max_div_weights = backtest_portfolio(data, betas, maximum_diversification_portfolio)
# risk_parity_returns, risk_parity_weights = backtest_portfolio(data, betas, risk_parity_portfolio)
# value_weighted_returns, value_weighted_weights = backtest_portfolio(data, betas, value_weighted_portfolio)
# equally_weighted_returns, equally_weighted_weights = backtest_portfolio(data, betas, equally_weighted_portfolio)

# # Evaluate performance
# cumulative_returns = []
# cumulative_returns.append(evaluate_performance(min_risk_returns, data['90 Day Bill Returns'].iloc[lookback:])[0])
# cumulative_returns.append(evaluate_performance(max_div_returns, data['90 Day Bill Returns'].iloc[lookback:])[0])
# cumulative_returns.append(evaluate_performance(risk_parity_returns, data['90 Day Bill Returns'].iloc[lookback:])[0])
# cumulative_returns.append(evaluate_performance(value_weighted_returns, data['90 Day Bill Returns'].iloc[lookback:])[0])
# cumulative_returns.append(evaluate_performance(equally_weighted_returns, data['90 Day Bill Returns'].iloc[lookback:])[0])

# # Plot the cumulative returns for comparison
# plot_cumulative_returns(cumulative_returns, labels=["Min Risk", "Max Diversification", "Risk Parity", "Value Weighted", "Equally Weighted"])



C:\Users\fz28\AppData\Local\Temp\ipykernel_3924\2392136494.py:27: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data['Stock Return'] = data['Monthly Total Return'].pct_change()


KeyError: 'Market Return'